# Training of PigNPi

In [46]:
import math
import pickle as pkl

import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch_geometric.loader import DataLoader

from model import GN_force_SiLU

torch.set_default_dtype(torch.float64)

In [33]:
with open('data.pkl', 'rb') as f:
    simulation_data = pkl.load(f)

In [34]:
vel = simulation_data["velocity"]
num_experiments, simulation_steps, num_nodes, dim = vel.shape

In [35]:
# edge_list and edge_feature
edge_list = []
edge_feature = []
for this_edge in simulation_data["edge_list"]:
    edge_list.append([this_edge[0], this_edge[1] ] ) # edge from i to j
    edge_list.append([this_edge[1], this_edge[0] ] ) # edge from j to i
edge_list = np.array(edge_list) # shape [num_edge, 2]

In [36]:
node_drag = simulation_data["drag_coefficient"]

In [37]:
node_feature = np.concatenate((simulation_data["position"], node_drag), axis=-1)
print(node_feature.shape)

(2, 1001, 10, 3)


In [38]:
# for flattening first two dimensions
node_feature_flatten = []
vel_flatten = []
for i in range(num_experiments):
    for j in range(simulation_steps):
        node_feature_flatten.append(node_feature[i, j] )
        vel_flatten.append(vel[i,j])

In [39]:
training_node_X = np.array(node_feature_flatten)
training_node_Y = np.array(vel_flatten)

In [40]:
# model configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

aggr = 'add'
hidden = 300
msg_dim = dim

# n_f = data.shape[3]
n_f = 2 * dim + 2 # node position, node velocity, node charge, node mass
model = GN_force_SiLU(n_node = num_nodes, n_f=n_f, msg_dim=msg_dim, ndim=dim, hidden=hidden, aggr=aggr).to(device)

In GN_force_SiLU, n_node is : 10


In [41]:
edge_list = np.transpose(edge_list)
print("edge_list shape: {}".format(edge_list.shape) )
print(edge_list.dtype)

edge_list shape: (2, 90)
int64


In [47]:
batch_size = 32
training_example_number = len(training_node_X)
trainloader = DataLoader(
    [Data(
        x = torch.from_numpy(training_node_X[i]),
        edge_index = torch.tensor(edge_list, dtype=torch.long),
        y = torch.from_numpy(training_node_Y[i]) )  for i in range(training_example_number ) ],
    batch_size=batch_size,
    shuffle=True
    )
